In [7]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_parquet('../../data/company/uk-company-similarity.parquet')
df['activity_code'] = df['activity_code'].astype(str)
df

,id,lat,long,activity_code
index,,,,
0,UK14774138,50.381220,-4.160850,62012
1,UK14794344,51.429590,-0.547460,82990
2,UK14799610,51.747310,-0.313780,41100
3,UK14804001,55.206640,-6.523650,47290
4,UK14804569,53.086690,-2.444910,53201
...,...,...,...,...
19276723,UK14580535,56.458705,-2.994404,None
19276724,UK14580555,55.950773,-3.205582,None
19276725,UK14580603,53.587020,-0.650410,82990


In [9]:
df['lat'] = df['lat'].fillna(df['lat'].mean())
df['long'] = df['long'].fillna(df['long'].mean())
df

,id,lat,long,activity_code
index,,,,
0,UK14774138,50.381220,-4.160850,62012
1,UK14794344,51.429590,-0.547460,82990
2,UK14799610,51.747310,-0.313780,41100
3,UK14804001,55.206640,-6.523650,47290
4,UK14804569,53.086690,-2.444910,53201
...,...,...,...,...
19276723,UK14580535,56.458705,-2.994404,None
19276724,UK14580555,55.950773,-3.205582,None
19276725,UK14580603,53.587020,-0.650410,82990


In [ ]:
df = pd.get_dummies(data=df, columns=['activity_code'], dummy_na=True)
df